In [1]:
import pathlib as pl
import numpy as np
import pandas as pd
import os
from preprocessing.noise_removal import preprocess_string
import ast

In [28]:
#path = (pl.Path(os.path.abspath('')).parent.resolve() / "data_files/test.tsv")
path = (pl.Path(os.path.abspath('')).parent.resolve() / "data_files/train.tsv")


In [29]:
df = pd.read_csv(path, delimiter='\t')

df

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10234,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10235,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10236,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10237,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...


In [69]:
new_df = pd.DataFrame()

In [70]:
new_df = pd.DataFrame({
       'type': df['false'],
       'words': df['Says the Annies List political group supports third-trimester abortions on demand.']
}) 

In [79]:
print(new_df['type'].value_counts())

reliable    3638
fake        2833
Name: type, dtype: int64


In [72]:
#viewing unique columns in dataframe
new_df['type'].unique()
new_df['orig_type'] = new_df['type'] # keeping orig type for later
new_df['id'] = [i for i in range(len(new_df))] # adding id column
new_df

,type,words,orig_type,id
0,half-true,When did the decline of coal start? It started...,half-true,0
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",mostly-true,1
2,false,Health care reform legislation is likely to ma...,false,2
3,half-true,The economic turnaround started at the end of ...,half-true,3
4,true,The Chicago Bears have had more starting quart...,true,4
...,...,...,...,...
10234,mostly-true,There are a larger number of shark attacks in ...,mostly-true,10234
10235,mostly-true,Democrats have now become the party of the [At...,mostly-true,10235
10236,half-true,Says an alternative to Social Security that op...,half-true,10236
10237,false,On lifting the U.S. Cuban embargo and allowing...,false,10237


In [73]:
# we disregard the middle labels
new_df['type'] = new_df['type'].map({'true': 'reliable', 'mostly-true': 'reliable', 'false': 'fake', 'pants-fire': 'fake'})
# 'half-true': 'reliable', 'barely-true': 'fake',
new_df

,type,words,orig_type,id
0,NaN,When did the decline of coal start? It started...,half-true,0
1,reliable,"Hillary Clinton agrees with John McCain ""by vo...",mostly-true,1
2,fake,Health care reform legislation is likely to ma...,false,2
3,NaN,The economic turnaround started at the end of ...,half-true,3
4,reliable,The Chicago Bears have had more starting quart...,true,4
...,...,...,...,...
10234,reliable,There are a larger number of shark attacks in ...,mostly-true,10234
10235,reliable,Democrats have now become the party of the [At...,mostly-true,10235
10236,NaN,Says an alternative to Social Security that op...,half-true,10236
10237,fake,On lifting the U.S. Cuban embargo and allowing...,false,10237


In [74]:
content_len = new_df['words'].apply(len)
print(content_len.sum()/ len(content_len))

new_df['words'] = new_df['words'].apply(preprocess_string)

new_df = new_df.dropna(subset =['type'], how = 'any', axis = 0)
new_df

106.92118370934662


,type,words,orig_type,id
1,reliable,"{'hillari': 1, 'clinton': 1, 'agre': 1, 'with'...",mostly-true,1
2,fake,"{'health': 1, 'care': 1, 'reform': 1, 'legisl'...",false,2
4,reliable,"{'the': 4, 'chicago': 1, 'bear': 1, 'have': 1,...",true,4
8,reliable,"{'say': 1, 'gop': 1, 'primari': 1, 'oppon': 1,...",mostly-true,8
9,reliable,"{'for': 1, 'the': 4, 'first': 1, 'time': 1, 'i...",mostly-true,9
...,...,...,...,...
10232,fake,"{'mayor': 1, 'fung': 1, 'want': 1, 'to': 2, 'p...",pants-fire,10232
10234,reliable,"{'there': 2, 'are': 2, 'a': 1, 'larger': 1, 'n...",mostly-true,10234
10235,reliable,"{'democrat': 1, 'have': 1, 'now': 1, 'becom': ...",mostly-true,10235
10237,fake,"{'on': 1, 'lift': 1, 'the': 1, 'us': 1, 'cuban...",false,10237


In [64]:
bow_content_len = new_df['words'].apply(lambda x: len(x.keys()))
print(bow_content_len.sum()/ len(bow_content_len))
#print(new_df['type'].value_counts())

1        18
2        12
4        23
8        20
9        16
         ..
10232    31
10234    14
10235    12
10237    11
10238    26
Name: words, Length: 6471, dtype: int64
15.963684129191778


In [25]:
csv_path = (pl.Path(os.path.abspath('')).parent.resolve() / 'data_files/LIAR.csv')

new_df.to_csv(csv_path, index=False)

In [26]:
print(csv_path)

/Users/victor/Documents/Uddannelse/Repos/fake-news/data_files/LIAR.csv
